In [1]:
# ============================================================
# SETUP AND DEPENDENCIES
# ============================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, Matern, WhiteKernel
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

print("Week 11 ML Models and Strategy")
print("="*60)

# ============================================================
# DATA LOADING (WEEKS 1-10)
# ============================================================

# All historical data through Week 10
data_w10 = {
    'F1': {
        'X': np.array([[0.10,0.10],[0.12,0.08],[0.21,0.11],[0.14,0.14],
                       [0.08,0.08],[0.45,0.45],[0.48,0.48],[0.405,0.428],
                       [0.403994,0.424817],[0.403994,0.424817]]),
        'y': np.array([0.0,0.0,0.0,0.0,0.0,0.0128,0.000008,0.462531,
                       0.457432,0.457432])
    },
    'F2': {
        'X': np.array([[0.10,0.10],[0.12,0.08],[0.21,0.11],[0.14,0.14],
                       [0.08,0.08],[0.111,0.100],[0.11,0.10],[0.111,0.100],
                       [0.100042,0.103196],[0.100042,0.103196]]),
        'y': np.array([0.0892,0.0705,0.0295,0.0150,0.0463,0.1300,0.0468,
                       -0.0246,0.033044,0.006300])
    },
    'F3': {
        'X': np.array([[0.80,0.80,0.80],[0.95,0.95,0.95],[0.98,0.99,0.87],
                       [0.948885,0.965632,0.808397],[1.01,1.01,0.82],
                       [0.928,0.832,0.004],[0.99,0.99,0.99],
                       [0.944,0.965,0.807],[0.954416,0.966710,0.808259],
                       [0.954416,0.966710,0.808259]]),
        'y': np.array([-0.1055,-0.0919,-0.0856,-0.0786,-1.1543,-0.1161,
                       -0.427251,-0.088593,-0.086019,-0.089657])
    },
    'F4': {
        'X': np.array([[0.5,0.5,0.5,0.5],[0.3,0.3,0.3,0.3],
                       [0.44,0.29,0.35,1.25],[0.51,0.60,0.57,0.01],
                       [0.66,0.30,0.30,0.36],[0.2,0.2,0.95,0.4],
                       [0.65,0.65,0.65,0.65],[0.498,0.502,0.500,0.500],
                       [0.488703,0.491179,0.485028,0.486178],
                       [0.488703,0.491179,0.485028,0.486178]]),
        'y': np.array([-3.986,-4.306,-30.129,-12.492,-7.262,-19.009,
                       -15.158,-3.9857,-4.430389,-4.430389])
    },
    'F5': {
        'X': np.array([[0.30,0.30,0.30,0.30],[0.28,0.32,0.30,0.29],
                       [0.344822,0.264687,0.374156,0.203902],
                       [0.196828,0.320017,0.300,0.289958],
                       [0.99,0.90,0.98,0.93],[0.985,0.905,0.975,0.925],
                       [1.0,0.853,1.0,0.977],[0.855,0.852,1.000,0.979],
                       [1.0,0.830999,1.0,0.988351],
                       [1.0,0.830999,1.0,0.988351]]),
        'y': np.array([136.85,137.29,131.78,140.74,5549.45,5398.58,
                       6158.08,4415.99,6117.763,6117.763])
    },
    'F6': {
        'X': np.array([[0.75,0.75,0.75,0.75,0.75],[0.3,0.3,0.3,0.3,0.3],
                       [0.49,0.02,0.45,0.40,0.32],[0.69,0.001,0.04,0.001,0.001],
                       [0.26,0.18,0.50,0.48,0.41],[0.1,0.1,0.7,0.7,0.6],
                       [0.15,0.15,0.50,0.50,0.70],[0.258,0.178,0.501,0.482,0.412],
                       [0.245246,0.162093,0.507258,0.481850,0.418363],
                       [0.245246,0.162093,0.507258,0.481850,0.418363]]),
        'y': np.array([-1.521,-1.139,-1.123,-2.067,-1.092,-1.231,-1.5517,
                       -1.064064,-1.068886,-1.163206])
    },
    'F7': {
        'X': np.array([[1.0,1.0,1.0,1.0,1.0,1.0],[0.2,0.2,0.2,0.2,0.2,0.2],
                       [0.21,0.19,0.21,0.19,0.17,0.19],
                       [0.08,0.32,0.15,0.28,0.41,0.27],
                       [0.05,0.50,0.25,0.20,0.15,0.85],
                       [0.06,0.48,0.25,0.20,0.40,0.75],
                       [0.038,0.462,0.239,0.171,0.378,0.734],
                       [0.039,0.463,0.240,0.172,0.379,0.742],
                       [0.022018,0.448224,0.257401,0.152511,0.397105,0.694372],
                       [0.022018,0.448224,0.257401,0.152511,0.397105,0.694372]]),
        'y': np.array([0.000034,0.408,0.347,0.568,0.836,1.435,1.478289,
                       1.463533,1.431577,1.431577])
    },
    'F8': {
        'X': np.array([[0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1],
                       [0.12,0.09,0.11,0.10,0.08,0.13,0.11,0.09],
                       [0.29,0.25,0.02,0.29,0.14,0.22,0.25,0.30],
                       [1.0,0.001,1.0,0.001,0.001,1.0,1.0,0.001],
                       [0.05,0.25,0.25,0.25,0.25,0.25,0.05,0.05],
                       [0.18,0.15,0.20,0.15,0.25,0.15,0.15,0.18],
                       [0.177,0.194,0.170,0.194,0.294,0.143,0.109,0.208],
                       [0.179,0.196,0.172,0.196,0.292,0.145,0.111,0.210],
                       [0.205880,0.195099,0.195658,0.188440,0.323931,0.147753,
                        0.081536,0.213205],
                       [0.205880,0.195099,0.195658,0.188440,0.323931,0.147753,
                        0.081536,0.213205]]),
        'y': np.array([9.542,9.554,9.548,4.180,9.643,9.676,9.692075,
                       9.691885,9.680707,9.680707])
    }
}

print("Data loaded: 10 weeks of history for all 8 functions\n")

# ============================================================
# F1 (2D): GAUSSIAN PROCESS - EMPIRICAL TREND CONTINUATION
# ============================================================

print("="*60)
print("F1 (2D): GAUSSIAN PROCESS WITH EMPIRICAL TREND")
print("="*60)

X_f1 = data_w10['F1']['X']
y_f1 = data_w10['F1']['y']

# Remove zeros for better GP fit
nonzero_mask = y_f1 > 0
X_f1_nz = X_f1[nonzero_mask]
y_f1_nz = y_f1[nonzero_mask]

print(f"\nTraining on non-zero data: {len(y_f1_nz)} points")
print(f"Output range: {y_f1_nz.min():.6f} to {y_f1_nz.max():.6f}")

# GP with Matern kernel
kernel_f1 = ConstantKernel(1.0) * Matern(
    length_scale=np.ones(2),
    length_scale_bounds=(1e-3, 10),
    nu=2.5
)

gp_f1 = GaussianProcessRegressor(
    kernel=kernel_f1,
    n_restarts_optimizer=10,
    alpha=1e-6,
    normalize_y=True
)

gp_f1.fit(X_f1_nz, y_f1_nz)

print("\nHistorical trend:")
print("  W6: [0.45, 0.45] -> 0.0128")
print("  W7: [0.48, 0.48] -> 0.000008 (overshot, declined)")
print("  W8: [0.405, 0.428] -> 0.462531 (BEST)")
print("  W9: [0.404, 0.425] -> 0.457432 (slight decline)")

print("\nEmpirical pattern: Peak around [0.405, 0.428]")
print("Strategy: Micro-step upward to test if peak extends")

# Test candidates
candidates_f1 = {
    'W8 repeat': [0.405, 0.428],
    'Micro-step up': [0.410, 0.433],
    'Larger step up': [0.415, 0.438]
}

print("\nGP Predictions:")
for name, candidate in candidates_f1.items():
    pred_mean, pred_std = gp_f1.predict([candidate], return_std=True)
    print(f"  {name}: {candidate}")
    print(f"    Predicted: {pred_mean[0]:.6f} +/- {pred_std[0]:.6f}")

print("\nDECISION: Micro-step up [0.410, 0.433]")
print("Rationale: GP shows slight positive gradient, test empirically")

w11_f1_input = [0.410, 0.433]
print(f"\nFinal F1 input: {w11_f1_input}")

# ============================================================
# F2 (2D): NO MODEL - STOCHASTIC NOISE DOMINATES
# ============================================================

print("\n" + "="*60)
print("F2 (2D): NO MODEL - STOCHASTIC")
print("="*60)

X_f2 = data_w10['F2']['X']
y_f2 = data_w10['F2']['y']

print("\nNoise Analysis:")
print("  [0.111, 0.100] -> W5: 0.1300, W7: -0.0246, W9: 0.033, W10: 0.0063")
print("  Variance: 80% on identical inputs")
print("  All modeling attempts failed (GP, RF, Bayesian Ridge)")

print("\nSTRATEGY: Abandon modeling")
print("Use best historical value from Week 5")

w11_f2_input = [0.111, 0.101]
print(f"\nFinal F2 input: {w11_f2_input}")
print("Rationale: Near W5 best, tiny perturbation for portal acceptance")

# ============================================================
# F3 (3D): GAUSSIAN PROCESS WITH ARD - GRADIENT FOLLOWING
# ============================================================

print("\n" + "="*60)
print("F3 (3D): GAUSSIAN PROCESS WITH ARD AND NOISE KERNEL")
print("="*60)

X_f3 = data_w10['F3']['X']
y_f3 = data_w10['F3']['y']

# Remove outliers (constraint violations)
valid_mask = y_f3 > -0.5
X_f3_valid = X_f3[valid_mask]
y_f3_valid = y_f3[valid_mask]

print(f"\nTraining on valid data: {len(y_f3_valid)} points")
print(f"Output range: {y_f3_valid.min():.6f} to {y_f3_valid.max():.6f}")
print("Goal: Minimize (get closer to zero)")

# GP with noise kernel (4% variance from W10 analysis)
kernel_f3 = (ConstantKernel(1.0) * Matern(
    length_scale=np.ones(3),
    length_scale_bounds=(1e-3, 10),
    nu=2.5
) + WhiteKernel(noise_level=0.01))

gp_f3 = GaussianProcessRegressor(
    kernel=kernel_f3,
    n_restarts_optimizer=15,
    alpha=1e-6,
    normalize_y=True
)

gp_f3.fit(X_f3_valid, y_f3_valid)

# Extract ARD length scales
length_scales_f3 = gp_f3.kernel_.k1.k2.length_scale
print("\nARD Length Scales:")
for i, ls in enumerate(length_scales_f3):
    print(f"  x{i+1}: {ls:.3f}")

print("\nHistorical best: W4 [0.949, 0.966, 0.808] -> -0.0786")
print("Strategy: Increase x3 based on GP gradient")

# Test candidates
candidates_f3 = {
    'W4 repeat': [0.949, 0.966, 0.808],
    'Small x3 increase': [0.949, 0.966, 0.820],
    'Larger x3 increase': [0.949, 0.966, 0.830]
}

print("\nGP Predictions:")
for name, candidate in candidates_f3.items():
    pred_mean, pred_std = gp_f3.predict([candidate], return_std=True)
    print(f"  {name}: {candidate}")
    print(f"    Predicted: {pred_mean[0]:.6f} +/- {pred_std[0]:.6f}")

print("\nDECISION: [0.949, 0.966, 0.830]")
print("Rationale: GP shows positive gradient toward higher x3")

w11_f3_input = [0.949, 0.966, 0.830]
print(f"\nFinal F3 input: {w11_f3_input}")

# ============================================================
# F4 (4D): QUADRATIC BOWL - ALREADY AT OPTIMUM
# ============================================================

print("\n" + "="*60)
print("F4 (4D): QUADRATIC BOWL - CENTER OPTIMUM")
print("="*60)

X_f4 = data_w10['F4']['X']
y_f4 = data_w10['F4']['y']

print("\nBest result: W8 [0.498, 0.502, 0.500, 0.500] -> -3.9857")
print("Function: Symmetric quadratic bowl")
print("Strategy: Stay near center, test slight asymmetry")

# Simple test: slight asymmetry
w11_f4_input = [0.501, 0.499, 0.501, 0.499]
print(f"\nFinal F4 input: {w11_f4_input}")
print("Rationale: Test if slight asymmetry improves over perfect center")

# ============================================================
# F5 (4D): EMPIRICAL GAP TESTING - x4=1.0
# ============================================================

print("\n" + "="*60)
print("F5 (4D): EMPIRICAL GAP ANALYSIS - THE BIG BET")
print("="*60)

X_f5 = data_w10['F5']['X']
y_f5 = data_w10['F5']['y']

# High-regime only
high_mask = y_f5 > 1000
X_f5_high = X_f5[high_mask]
y_f5_high = y_f5[high_mask]

print(f"\nTraining on high-regime: {len(y_f5_high)} points")
print(f"Output range: {y_f5_high.min():.2f} to {y_f5_high.max():.2f}")

# Analyze x4 values tested
x4_tested = X_f5_high[:, 3]
print(f"\nx4 values tested in high regime: {sorted(set(x4_tested))}")
print("EMPIRICAL GAP: 1.0 has never been tested!")
print("  Tested: 0.925, 0.977, 0.979, 0.988")
print("  Gap at boundary: 0.988 to 1.0")

# GP with ARD
kernel_f5 = ConstantKernel(1.0) * Matern(
    length_scale=np.ones(4),
    length_scale_bounds=(1e-3, 10),
    nu=2.5
)

gp_f5 = GaussianProcessRegressor(
    kernel=kernel_f5,
    n_restarts_optimizer=15,
    alpha=1e-6,
    normalize_y=True
)

gp_f5.fit(X_f5_high, y_f5_high)

# ARD analysis
length_scales_f5 = gp_f5.kernel_.k2.length_scale
print("\nARD Length Scales:")
variables = ['x1', 'x2', 'x3', 'x4']
for var, ls in zip(variables, length_scales_f5):
    importance = 1.0 / (ls + 1e-10)
    status = "CRITICAL" if ls < 0.1 else "Moderate" if ls < 0.3 else "Exploratory"
    print(f"  {var}: {ls:.3f} ({status})")

print("\nInterpretation:")
print("  x1, x3: CRITICAL boundaries (must = 1.0)")
print("  x2: Moderate sensitivity (optimum at 0.853)")
print("  x4: Longest length scale (safe to explore boundary)")

# Predict x4=1.0
candidate_f5 = [1.0, 0.853, 1.0, 1.0]
pred_mean, pred_std = gp_f5.predict([candidate_f5], return_std=True)

print(f"\nCandidate: {candidate_f5}")
print(f"GP Prediction: {pred_mean[0]:.2f} +/- {pred_std[0]:.2f}")
print(f"Best so far: W7 6158.08")
print(f"Predicted improvement: {pred_mean[0] - 6158.08:.2f}")

print("\nDECISION: Test x4=1.0 based on empirical gap")
print("Rationale: Obvious untested boundary, GP uncertainty high")

w11_f5_input = [1.0, 0.853, 1.0, 1.0]
print(f"\nFinal F5 input: {w11_f5_input}")

# ============================================================
# F6 (5D): GP WITH NOISE - TEST LOWER x1
# ============================================================

print("\n" + "="*60)
print("F6 (5D): GAUSSIAN PROCESS WITH NOISE KERNEL")
print("="*60)

X_f6 = data_w10['F6']['X']
y_f6 = data_w10['F6']['y']

print(f"\nTraining on {len(y_f6)} points")
print(f"Output range: {y_f6.min():.6f} to {y_f6.max():.6f}")
print("Goal: Minimize (less negative)")

# GP with noise kernel (9% variance from W10)
kernel_f6 = (ConstantKernel(1.0) * Matern(
    length_scale=np.ones(5),
    length_scale_bounds=(1e-3, 10),
    nu=2.5
) + WhiteKernel(noise_level=0.05))

gp_f6 = GaussianProcessRegressor(
    kernel=kernel_f6,
    n_restarts_optimizer=15,
    alpha=1e-6,
    normalize_y=True
)

gp_f6.fit(X_f6, y_f6)

print("\nHistorical best: W8 [0.245, 0.162, 0.507, 0.482, 0.418] -> -1.064")
print("Strategy: Test if x1 lower improves")

# Test candidates
candidates_f6 = {
    'W8 best': [0.245, 0.162, 0.507, 0.482, 0.418],
    'x1 lower': [0.200, 0.162, 0.507, 0.482, 0.418]
}

print("\nGP Predictions:")
for name, candidate in candidates_f6.items():
    pred_mean, pred_std = gp_f6.predict([candidate], return_std=True)
    print(f"  {name}: {candidate}")
    print(f"    Predicted: {pred_mean[0]:.6f} +/- {pred_std[0]:.6f}")

print("\nDECISION: Test x1=0.200")
print("Rationale: GP suggests x1 lower might improve")

w11_f6_input = [0.200, 0.162, 0.507, 0.482, 0.418]
print(f"\nFinal F6 input: {w11_f6_input}")

# ============================================================
# F7 (6D): EMPIRICAL PATTERN - FILL GAP AT x6
# ============================================================

print("\n" + "="*60)
print("F7 (6D): EMPIRICAL PATTERN ANALYSIS")
print("="*60)

X_f7 = data_w10['F7']['X']
y_f7 = data_w10['F7']['y']

# Analyze x6 pattern
print("\nHistorical x6 performance:")
for x, y in zip(X_f7[-6:], y_f7[-6:]):
    print(f"  x6={x[5]:.3f}: output={y:.6f}")

print("\nPattern observed:")
print("  x6=0.694 -> 1.432")
print("  x6=0.734 -> 1.478 (PEAK)")
print("  x6=0.742 -> 1.464 (decline)")
print("  x6=0.750 -> would continue declining")

print("\nEmpirical gap: Between 0.734 and 0.799")
print("Strategy: Test x6=0.760 to verify decline continues")

w11_f7_input = [0.022, 0.448, 0.240, 0.153, 0.397, 0.760]
print(f"\nFinal F7 input: {w11_f7_input}")
print("Rationale: Fill empirical gap, expect decline from peak")

# ============================================================
# F8 (8D): CONVERGED - MINOR PERTURBATION
# ============================================================

print("\n" + "="*60)
print("F8 (8D): CONVERGED - MINOR REFINEMENT")
print("="*60)

X_f8 = data_w10['F8']['X']
y_f8 = data_w10['F8']['y']

print("\nBest: W7 [0.177, 0.194, 0.170, 0.194, 0.294, 0.143, 0.109, 0.208]")
print("      -> 9.692075")
print("W8-W10 all within 0.01 of W7 best")
print("Function appears converged")

print("\nStrategy: Tiny perturbations from W9 best")

w11_f8_input = [0.207, 0.196, 0.197, 0.189, 0.325, 0.149, 0.083, 0.214]
print(f"\nFinal F8 input: {w11_f8_input}")
print("Rationale: Function converged, minor exploration only")

# ============================================================
# WEEK 11 FINAL INPUTS SUMMARY
# ============================================================

print("\n" + "="*60)
print("WEEK 11 FINAL INPUTS - ALL FUNCTIONS")
print("="*60)

w11_inputs = {
    'F1': w11_f1_input,
    'F2': w11_f2_input,
    'F3': w11_f3_input,
    'F4': w11_f4_input,
    'F5': w11_f5_input,
    'F6': w11_f6_input,
    'F7': w11_f7_input,
    'F8': w11_f8_input
}

print("\nFunction | Input | Strategy")
print("-" * 60)
for fn, inp in w11_inputs.items():
    strategy_map = {
        'F1': 'GP + Empirical trend',
        'F2': 'No model (stochastic)',
        'F3': 'GP with noise kernel',
        'F4': 'Quadratic bowl (center)',
        'F5': 'EMPIRICAL GAP (x4=1.0)',
        'F6': 'GP with noise kernel',
        'F7': 'Empirical pattern',
        'F8': 'Converged (minor tweak)'
    }
    print(f"{fn}      | {inp} | {strategy_map[fn]}")

# Portal format
print("\n" + "="*60)
print("PORTAL FORMAT (6 decimal places)")
print("="*60)

for fn, inp in w11_inputs.items():
    formatted = '-'.join([f"{v:.6f}" for v in inp])
    print(f"{fn}: {formatted}")

# ============================================================
# WEEK 11 ACTUAL RESULTS
# ============================================================

print("\n" + "="*60)
print("WEEK 11 ACTUAL RESULTS")
print("="*60)

w11_results = {
    'F1': 0.525860,
    'F2': 0.068939,
    'F3': -0.061424,
    'F4': -3.985737,
    'F5': 6526.44,
    'F6': -1.185411,
    'F7': 1.315567,
    'F8': 9.681707
}

total_w11 = sum(w11_results.values())

print("\nFunction | Predicted | Actual | Result")
print("-" * 60)

# F1
pred_f1, _ = gp_f1.predict([[0.410, 0.433]], return_std=True)
result_f1 = "SUCCESS" if w11_results['F1'] > 0.462531 else "DECLINE"
print(f"F1      | {pred_f1[0]:.6f} | {w11_results['F1']:.6f} | {result_f1}")

# F2
print(f"F2      | No model | {w11_results['F2']:.6f} | Noise")

# F3
pred_f3, _ = gp_f3.predict([[0.949, 0.966, 0.830]], return_std=True)
result_f3 = "SUCCESS" if w11_results['F3'] > -0.0786 else "DECLINE"
print(f"F3      | {pred_f3[0]:.6f} | {w11_results['F3']:.6f} | {result_f3}")

# F4
print(f"F4      | ~-3.986 | {w11_results['F4']:.6f} | Similar")

# F5
pred_f5, _ = gp_f5.predict([[1.0, 0.853, 1.0, 1.0]], return_std=True)
improvement_f5 = w11_results['F5'] - 6158.08
print(f"F5      | {pred_f5[0]:.2f} | {w11_results['F5']:.2f} | BREAKTHROUGH (+{improvement_f5:.2f})")

# F6
pred_f6, _ = gp_f6.predict([[0.200, 0.162, 0.507, 0.482, 0.418]], return_std=True)
result_f6 = "WORSE" if w11_results['F6'] < -1.064 else "BETTER"
print(f"F6      | {pred_f6[0]:.6f} | {w11_results['F6']:.6f} | {result_f6}")

# F7
result_f7 = "DECLINE" if w11_results['F7'] < 1.478 else "IMPROVE"
print(f"F7      | Decline | {w11_results['F7']:.6f} | {result_f7}")

# F8
print(f"F8      | ~9.68 | {w11_results['F8']:.6f} | Stable")

print(f"\nTOTAL WEEK 11: {total_w11:.2f}")
print(f"Previous best (W7): 6164.71")
print(f"Improvement: +{total_w11 - 6164.71:.2f} points")

print("\n" + "="*60)
print("KEY INSIGHTS FROM WEEK 11")
print("="*60)

print("\nSUCCESSES:")
print("  F5: Empirical gap (x4=1.0) gained +368 points")
print("  F1: Empirical trend continuation worked (+13.7%)")
print("  F3: GP gradient following improved 21.9%")
print("  F4: Already at optimum, maintained")

print("\nFAILURES:")
print("  F6: GP suggested x1=0.200, made it worse")
print("  F7: Empirical gap confirmed decline (peak at 0.734)")
print("  F2: Stochastic noise (as expected)")

print("\nCRITICAL LEARNING:")
print("  Empirical gap-filling (x4=1.0) outperformed GP optimization")
print("  GP accurate for smooth interpolation (F3)")
print("  GP failed at boundaries/extrapolation (F6, F7)")
print("  Trust empirical patterns over model predictions at edges")

Week 11 ML Models and Strategy
Data loaded: 10 weeks of history for all 8 functions

F1 (2D): GAUSSIAN PROCESS WITH EMPIRICAL TREND

Training on non-zero data: 5 points
Output range: 0.000008 to 0.462531

Historical trend:
  W6: [0.45, 0.45] -> 0.0128
  W7: [0.48, 0.48] -> 0.000008 (overshot, declined)
  W8: [0.405, 0.428] -> 0.462531 (BEST)
  W9: [0.404, 0.425] -> 0.457432 (slight decline)

Empirical pattern: Peak around [0.405, 0.428]
Strategy: Micro-step upward to test if peak extends

GP Predictions:
  W8 repeat: [0.405, 0.428]
    Predicted: 0.462526 +/- 0.000222
  Micro-step up: [0.41, 0.433]
    Predicted: 0.473651 +/- 0.015417
  Larger step up: [0.415, 0.438]
    Predicted: 0.456770 +/- 0.044127

DECISION: Micro-step up [0.410, 0.433]
Rationale: GP shows slight positive gradient, test empirically

Final F1 input: [0.41, 0.433]

F2 (2D): NO MODEL - STOCHASTIC

Noise Analysis:
  [0.111, 0.100] -> W5: 0.1300, W7: -0.0246, W9: 0.033, W10: 0.0063
  Variance: 80% on identical inputs
